In [ ]:
# Import necessary packages

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import LeakyReLU, Reshape, Conv2DTranspose

import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import random

In [ ]:
noise_dim = 100

In [ ]:
# Generator
def make_generator_model():
    model = Sequential()
    model.add(Dense(15, activation='relu', input_dim=noise_dim))
    model.add(Dense(2, activation='linear'))
    return model
generator = make_generator_model()
generator.summary()


In [ ]:
# Discriminator
def make_discriminator_model():
    model = Sequential()
    model.add(Dense(25, activation='relu',input_dim=2))
    model.add(Dense(1, activation='sigmoid'))
    return model

discriminator = make_discriminator_model()
discriminator.summary()

In [ ]:
# Optimizers and loss function
generator_optimizer = tf.keras.optimizers.Adam()
discriminator_optimizer = tf.keras.optimizers.Adam()
cross_entropy = tf.keras.losses.BinaryCrossentropy()

discriminator = make_discriminator_model()
generator = make_generator_model()

In [ ]:
# Generator of real points
import random
import math
def generate_real_points(n):
    points = []
    for i in range(n):
        x = random.random()*7-(7/2)
        #y = x #
        y = math.sin(x)
        points.append((x,y))
    points = np.array(points)
    return points

In [ ]:
# One step (epoch) of training
def do_step(batch_size):
    real_points = generate_real_points(batch_size)
    noise = tf.random.normal([batch_size, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # generate fake points uning noise
        generated_points = generator(noise, training=True) 
        # check real and fake points with discriminator
        real_output = discriminator(real_points, training=True) 
        fake_output = discriminator(generated_points, training=True)
        # discriminator loss - based on the number of wrong classification
        real_loss = cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
        disc_loss = (real_loss + fake_loss)/2 # avg of both losses
        # generator loss - based of the number of fake points recognized by discriminator
        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)

    # calculate gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables) 
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # apply gradients and update weights
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables)) 
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss,disc_loss

In [ ]:
# Visualization - generates 100 example real points and 100 example fake points and shows the plot
def show_results():
  print("Epoch",epoch,f"generator loss={gl:.3f},discriminator loss={dl:.3f}")
  num_examples_to_generate = 100
        
  seed = tf.random.normal([num_examples_to_generate, noise_dim])
  fake_points = generator(seed, training=False)
  plt.scatter(fake_points[:,0],fake_points[:,1] , label='fake')

  real_points = generate_real_points(num_examples_to_generate)
  plt.scatter(real_points[:,0],real_points[:,1], label='real')
  plt.legend()
  
  # red cross over each point classified as fake by the discriminator
  fake_prediction = discriminator.predict(fake_points)
  num_true_neg = 0
  for i in range(num_examples_to_generate):
    if fake_prediction[i]<0.5:
      plt.scatter(fake_points[i,0],fake_points[i,1],marker='.',color='red')
      num_true_neg += 1 

  real_prediction = discriminator.predict(real_points)
  num_false_neg = 0
  for i in range(num_examples_to_generate):
    if real_prediction[i]<0.5:
      plt.scatter(real_points[i,0],real_points[i,1],marker='.',color='red')
      num_false_neg += 1
  accuracy_real = (num_examples_to_generate-num_false_neg)/num_examples_to_generate
  accuracy_fake = (num_examples_to_generate-num_true_neg)/num_examples_to_generate
  # Accuracy of the discriminator
  print("Accuracy real",accuracy_real,"accuracy fake",accuracy_fake)
  plt.show()    

In [ ]:
# Main loop - run steps and visualize for every 100 epochs
epochs = 5000
for epoch in range(epochs):
    gl,dl = do_step(150)
    if epoch % 100 == 0:
       show_results()
print("Done")

In [ ]:
# test the generator
num_examples_to_generate = 100
    
seed = tf.random.normal([num_examples_to_generate, noise_dim])
fake_points = generator(seed, training=False)
plt.scatter(fake_points[:,0],fake_points[:,1] , label='fake')
plt.legend()
plt.show()  


In [ ]:
vals = [(x,y) for x in np.linspace(-4,4,100) for y in np.linspace(-4,4,100)]
vals = np.array(vals)
preds = discriminator.predict(vals)
plt.scatter(vals[:,0],vals[:,1] , c=preds[:])
plt.show()